# Deep Research: ETF Pairs Trading Optimization

**Navigation** : [Index](../../README.md) | [Retour](../README.md)

## Objectif

Maximiser le ratio de Sharpe pour la strategie de pairs trading sur ETF.

## Vue d'ensemble de la strategie

- **Selection des pairs** : Test de cointegration + filtre demi-vie + filtre volatilite
- **Entree** : Le spread s'ecarte de la moyenne (z-score > seuil)
- **Sortie** : Le spread revertent a la moyenne (z-score traverse 0)
- **Gestion du risque** : Stops par jambe desactives (preserver la neutralite du marche)

## Parametres actuels (a optimiser)

- `half_life_max` : 30 jours (filtre les pairs a reversion lente)
- `z_exit_threshold` : 0.0 (sortie a la moyenne)
- `stop_loss_per_leg` : False

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre le concept de cointegration et son application au pairs trading
2. Calculer la demi-vie de reversion a la moyenne (processus d'Ornstein-Uhlenbeck)
3. Implementer une selection de pairs multi-criteres (cointegration, volatilite, demi-vie)
4. Backtester une strategie de pairs trading avec z-score
5. Executer un grid search pour optimiser les parametres
6. Analyser la performance market-neutral de la strategie

### Prerequis

- Python 3.10+ avec pandas, numpy, statsmodels, yfinance
- Connaissance des statistiques (correlation, regression, tests statistiques)
- Comprehension du z-score et des processus stochastiques
- Familiarite avec les ETFs et le trading market-neutral

### Duree estimee : 75 minutes

## Questions de recherche

1. Quel est le seuil de demi-vie optimal pour la selection des pairs ?
2. Doit-on utiliser un seuil de sortie z-score different de 0 ?
3. Quel seuil de volatilite du spread maximise les rendements ajustes du risque ?
4. Quels pairs ETF ont les relations de cointegration les plus stables ?

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
from itertools import combinations

# ETF Universe
ETF_LIST = [
    'SPY',   # S&P 500
    'QQQ',   # NASDAQ-100
    'IWM',   # Russell 2000
    'DIA',   # Dow Jones
    'VTI',   # Total Market
    'XLE',   # Energy
    'XLK',   # Technology
    'XLF',   # Financial
    'XLV',   # Healthcare
    'XLY',   # Consumer Discretionary
]

print(f"ETF Universe: {len(ETF_LIST)} ETFs")
print(f"Possible pairs: {len(list(combinations(ETF_LIST, 2)))}")

In [ ]:
# Download ETF data
print("Downloading ETF data...")
etf_data = {}
for etf in ETF_LIST:
    try:
        df = yf.download(etf, start="2015-01-01", end="2025-02-18", progress=False)
        if not df.empty:
            etf_data[etf] = df['Close'].copy()
            if isinstance(etf_data[etf], pd.DataFrame):
                etf_data[etf] = etf_data[etf].iloc[:, 0]
            print(f"  {etf}: {len(etf_data[etf])} days")
    except Exception as e:
        print(f"  {etf}: Error - {e}")

print(f"\nSuccessfully downloaded {len(etf_data)} ETFs")

### Interpretation : Donnees historiques ETF

Le chargement des donnees ETF fournit la base pour l'analyse de cointegration et le backtesting.

| ETF | Description | Role typique dans les pairs |
|-----|-------------|----------------------------|
| SPY | S&P 500 | Reference large market |
| QQQ | NASDAQ-100 | Tech/Growth |
| IWM | Russell 2000 | Small cap |
| DIA | Dow Jones | Blue chips |
| VTI | Total Market | Broad diversification |
| XLK, XLF, XLV, XLY, XLE | Sector ETFs | Sector-specific plays |

**Points cles** :
- Les ETF sectoriels offrent des opportunits de pairs intra-sector (ex: XLK vs XLV)
- Les broad market ETFs (SPY, VTI) sont souvent cointegres entre eux
- La periode 2015-2025 couvre differents regimes de marche (bull, bear, COVID)
- La liquidite des ETF garantit des executions fiables pour le pairs trading

> **Note technique** : Pour la production sur QuantConnect, ces ETFs sont disponibles avec des donnees quotidiennes depuis 2000+. Le pairs trading fonctionne mieux avec des donnees quotidiennes ou hebdomadaires.

In [ ]:
# Calculate half-life of mean reversion
def calculate_half_life(spread):
    """Calculate half-life of mean reversion using OU process"""
    spread_lagged = spread[:-1]
    spread_current = spread[1:]
    
    # Remove NaN
    mask = ~(np.isnan(spread_lagged) | np.isnan(spread_current))
    spread_lagged = spread_lagged[mask]
    spread_current = spread_current[mask]
    
    if len(spread_lagged) < 10:
        return 999
    
    # Calculate lag-1 autocorrelation
    corr_lag1 = np.corrcoef(spread_lagged, spread_current)[0, 1]
    
    if np.isnan(corr_lag1) or corr_lag1 <= 0 or corr_lag1 >= 1:
        return 999
    
    # Half-life = -ln(2) / ln(corr)
    half_life = -np.log(2) / np.log(corr_lag1)
    return half_life

# Test with example pair (SPY-QQQ)
if 'SPY' in etf_data and 'QQQ' in etf_data:
    # Align data
    df = pd.DataFrame({'SPY': etf_data['SPY'], 'QQQ': etf_data['QQQ']}).dropna()
    
    # Calculate spread (price ratio)
    spread = df['SPY'] / df['QQQ']
    
    # Calculate half-life
    hl = calculate_half_life(spread.values)
    print(f"SPY-QQQ Half-life: {hl:.1f} days")
    
    # Calculate spread volatility
    spread_vol = spread.pct_change().std() * np.sqrt(252)
    print(f"SPY-QQQ Spread Volatility: {spread_vol*100:.2f}%")

### Interpretation : Demi-vie de reversion a la moyenne

La demi-vie mesure le temps necessaire pour que le spread revienne a sa moyenne apres une deviation.

| Concept | Formule | Interpretation |
|---------|---------|----------------|
| Half-life | -ln(2) / ln(corr_lag1) | Jours pour que l'ecart diminue de moitie |
| OU Process | dX = theta(mu - X)dt + sigma dW | Modele de retour a la moyenne |

**Points cles** :
- Une demi-vie courte (< 20 jours) indique une reversion rapide (ideal pour le trading)
- Une demi-vie longue (> 60 jours) indique une convergence lente (eviter)
- La volatilite du spread doit etre suffisante pour generer des signaux
- Le pair SPY-QQQ est un exemple typique de deux indices fortement correles

> **Note technique** : La demi-vie est calculee a partir de l'autocorrelation lag-1. Pour des estimations plus robustes, considerer la regression OLS sur le processus d'Ornstein-Uhlenbeck.

In [ ]:
# Select pairs based on cointegration and half-life
def select_pairs(etf_data, 
                 pvalue_threshold=0.05,
                 vol_threshold=0.01,
                 half_life_max=30):
    """
    Select cointegrated pairs with:
    - P-value < pvalue_threshold (strong cointegration)
    - Spread volatility > vol_threshold (sufficient deviation)
    - Half-life < half_life_max (reasonably fast reversion)
    """
    results = []
    
    for etf1, etf2 in combinations(etf_data.keys(), 2):
        # Align data
        df = pd.DataFrame({etf1: etf_data[etf1], etf2: etf_data[etf2]}).dropna()
        
        if len(df) < 252:
            continue
        
        try:
            # Cointegration test
            score, pvalue, _ = coint(df[etf1], df[etf2])
            
            # Calculate spread (hedge-adjusted)
            spread = df[etf1] - df[etf2] * (df[etf1].mean() / df[etf2].mean())
            
            # Spread volatility
            vol = spread.pct_change().std()
            
            # Half-life
            hl = calculate_half_life(spread.values)
            
            # Correlation
            corr = df[etf1].corr(df[etf2])
            
            if pvalue < pvalue_threshold and vol > vol_threshold and hl < half_life_max:
                results.append({
                    'etf1': etf1,
                    'etf2': etf2,
                    'pvalue': pvalue,
                    'correlation': corr,
                    'volatility': vol,
                    'half_life': hl
                })
        except Exception as e:
            continue
    
    return pd.DataFrame(results)

# Test current parameters
pairs_current = select_pairs(etf_data, half_life_max=30)
print(f"Pairs with half_life < 30 days: {len(pairs_current)}")
if not pairs_current.empty:
    print(pairs_current.sort_values('half_life').head(10).to_string())

### Interpretation : Selection des pairs cointegres

La selection des pairs combine trois criteres pour identifier les opportunitses de pairs trading.

| Critere | Description | Seuil |
|---------|-------------|-------|
| Cointegration | Test statistique de relation long terme | p-value < 0.05 |
| Volatilite du spread | Amplitude des deviations du spread | vol > 0.01 |
| Demi-vie | Vitesse de reversion a la moyenne | half-life < 30 jours |

**Points cles** :
- La cointegration garantit que les prix vont converger a long terme
- La demi-vie courte (< 30 jours) assure une reversion rapide
- La volatilite minimale assure des opportunites de trading exploitables
- Le nombre de pairs selectionnes depend de la severite des criteres

> **Note technique** : Le test de cointegration d'Engle-Granger est une methode simple mais limitee. Pour des analyses plus avancees, considerer le test de Johansen (cointegration multiple).

In [ ]:
# Backtest pairs trading strategy
def backtest_pairs(etf_data, pairs, z_entry=2.0, z_exit=0.0, lookback=20):
    """
    Simple backtest for pairs trading.
    
    Entry: When z-score > z_entry (or < -z_entry)
    Exit: When z-score crosses z_exit
    """
    if pairs.empty:
        return None
    
    total_returns = []
    
    for _, row in pairs.iterrows():
        etf1, etf2 = row['etf1'], row['etf2']
        
        # Get aligned data
        df = pd.DataFrame({etf1: etf_data[etf1], etf2: etf_data[etf2]}).dropna()
        
        # Calculate spread
        hedge_ratio = (df[etf1].mean() / df[etf2].mean())
        spread = df[etf1] - df[etf2] * hedge_ratio
        
        # Z-score
        spread_mean = spread.rolling(lookback).mean()
        spread_std = spread.rolling(lookback).std()
        z_score = (spread - spread_mean) / spread_std
        
        # Generate signals
        long_signal = (z_score > z_entry).astype(int) - (z_score < -z_entry).astype(int)
        
        # Exit when z_score crosses z_exit
        exit_long = (z_score <= z_exit) & (long_signal == 1)
        exit_short = (z_score >= -z_exit) & (long_signal == -1)
        
        # Calculate returns (simplified)
        ret1 = df[etf1].pct_change()
        ret2 = df[etf2].pct_change()
        
        # Pair return (long ETF1, short ETF2)
        pair_ret = ret1 - hedge_ratio * ret2
        
        # Strategy return
        strat_ret = pair_ret * long_signal.shift(1)
        total_returns.append(strat_ret)
    
    # Combine all pair returns
    combined = pd.concat(total_returns, axis=1).mean(axis=1)
    combined = combined.dropna()
    
    # Metrics
    sharpe = np.sqrt(252) * combined.mean() / combined.std() if combined.std() > 0 else 0
    total_return = (1 + combined).cumsum().iloc[-1]
    max_dd = (combined.cumsum() / combined.cumsum().cummax() - 1).min()
    
    return {
        'sharpe': sharpe,
        'total_return': total_return,
        'max_drawdown': max_dd
    }

# Test current parameters
result = backtest_pairs(etf_data, pairs_current, z_entry=2.0, z_exit=0.0)
if result:
    print("Current Parameters (half_life_max=30, z_exit=0.0):")
    print(f"  Sharpe: {result['sharpe']:.3f}")
    print(f"  Total Return: {result['total_return']*100:.1f}%")
    print(f"  Max Drawdown: {result['max_drawdown']*100:.1f}%")

### Interpretation : Performance du backtest

Le backtest simule la strategie de pairs trading sur les ETF selectionnes.

| Metrique | Valeur | Interpretation |
|----------|--------|----------------|
| Sharpe | A calculer | Ratio de Sharpe > 1 est considere bon |
| Total Return | A calculer | Performance cumulee sur la periode |
| Max Drawdown | A calculer | Perte maximale from peak |

**Points cles** :
- La strategie pairs trading est market-neutral : elle ne depend pas de la direction du marche
- Le Sharpe doit etre interprete en fonction du nombre de pairs (diversification)
- Le drawdown est limite par la construction market-neutral de la strategie
- Les resultats dependent fortement de la qualite de la selection des pairs

> **Note technique** : Le backtest simplifie ne tient pas compte des couts de transaction. En production, il faut considerer les commissions et l'impact de marche.

In [ ]:
# Grid search for optimal parameters
def grid_search_pairs(etf_data):
    results = []
    
    for hl_max in [15, 20, 25, 30, 40, 50]:
        # Select pairs with this half-life threshold
        pairs = select_pairs(etf_data, half_life_max=hl_max)
        
        if pairs.empty:
            continue
        
        for z_exit in [-0.5, 0.0, 0.5, 1.0]:
            for lookback in [10, 20, 30, 60]:
                result = backtest_pairs(etf_data, pairs, z_entry=2.0, z_exit=z_exit, lookback=lookback)
                
                if result:
                    results.append({
                        'half_life_max': hl_max,
                        'z_exit': z_exit,
                        'lookback': lookback,
                        'num_pairs': len(pairs),
                        'sharpe': result['sharpe'],
                        'total_return': result['total_return'],
                        'max_drawdown': result['max_drawdown']
                    })
    
    return pd.DataFrame(results)

print("Running grid search...")
grid_results = grid_search_pairs(etf_data)
grid_results = grid_results.sort_values('sharpe', ascending=False)

print("\nTop 10 Parameter Combinations:")
print(grid_results.head(10).to_string())

### Interpretation : Resultats du grid search

Le grid search explore l'espace des parametres pour identifier la combinaison optimale maximisant le ratio de Sharpe.

| Parametre | Plage testee | Impact sur la performance |
|-----------|--------------|---------------------------|
| `half_life_max` | 15-50 jours | Plus bas = pairs plus reactifs, moins de pairs |
| `z_exit` | -0.5 a 1.0 | Seuil de sortie : 0 = moyenne, >0 = plus conservateur |
| `lookback` | 10-60 jours | Fenetre de calcul du z-score |

**Points cles** :
- Le nombre de pairs disponibles diminue avec un seuil de demi-vie plus bas
- Un z_exit positif (0.5) reduit le whipsaw mais peut reduire le nombre de trades
- Un lookback court (10-20) est plus reactif mais plus sensible au bruit
- Le Sharpe doit etre interprete en fonction du nombre de pairs (diversification)

> **Note technique** : Les pairs ETF sont des actifs tres liquides avec des spreads etroits. L'impact des couts de transaction est donc limite, permettant des periodes de holding plus courtes.

## Conclusion : Recommandations de recherche

### Resume des resultats

L'analyse par grid search a permis d'identifier les parametres optimaux pour la strategie de pairs trading ETF.

| Parametre | Valeur optimale | Impact |
|-----------|-----------------|--------|
| HALF_LIFE_MAX | A determiner par grid search | Filtre les pairs a reversion lente |
| Z_EXIT_THRESHOLD | A determiner | Seuil de sortie (0 = moyenne) |
| LOOKBACK_PERIOD | A determiner | Fenetre de calcul du z-score |

### Points cles a retenir

1. **Demi-vie (Half-life)** : Les pairs avec une demi-vie courte (10-25 jours) revertent plus rapidement
2. **Seuil de sortie Z** : Un seuil positif (0.5) peut reduire le whipsaw mais reduit le nombre de trades
3. **Periode de lookback** : Une fenetre courte (10-20 jours) est plus reactive aux changements de regime
4. **Neutralite du marche** : La strategie pairs trading est market-neutral par construction

> **Note technique** : La selection des pairs doit etre reevaluee periodiquement (mensuellement ou trimestriellement) car les relations de cointegration peuvent evoluer avec le temps.

### Prochaines etapes

1. Appliquer les parametres trouves au code C# QuantConnect
2. Implementer la reselection periodique des pairs
3. Ajouter des filtres supplementaires (correlation minime, volume minimum)
4. Backtester sur QuantConnect avec les donnees reelles
5. Valider la robustesse avec walk-forward analysis